# Day 3- Multi-Model Integration: Using Gemini, DeepSeek & Grok with OpenAI Agents

### Summary
This session recaps how to create tools and handoffs within the OpenAI Agents SDK and then extends these concepts to integrate non-OpenAI models like Gemini, Deep Seek, and Llama 3. The key technique involves using OpenAI-compatible API endpoints, allowing the SDK to act as a universal client for various language models. The lesson also introduces the upcoming topics of structured outputs and guardrails, which are essential for building robust, predictable, and safe multi-agent systems.

### Highlights
- **Recap of Core Concepts**: The session revisits key mechanics from the previous lesson: using the `@tool` decorator to wrap functions, converting agents into tools with `as_tool`, and understanding the difference between tools (a function call that returns control) and handoffs (a permanent transfer of control to another agent).
- **Integrating Non-OpenAI Models**: The primary new concept is using the OpenAI Agents SDK with models other than OpenAI's. This is achieved by pointing the SDK to an OpenAI-compatible API endpoint (a `base_url`) for services like Gemini, Deep Seek, or Groq.
- **Client Instantiation for Other Models**: To connect to a non-OpenAI model, you must first instantiate an `AsyncOpenAI` client, providing it with the model's specific `base_url` and `api_key`. This configured client object is then used to create a model instance.
- **Model Object vs. Model String**: When creating an agent, passing a model name as a string (e.g., `"gpt-4-mini"`) defaults to the OpenAI API. To use a different model, you must pass an `OpenAIChatCompletionsModel` object that has been initialized with your custom client, effectively telling the agent which endpoint to communicate with.
- **Multi-Model Agent System**: The tutorial demonstrates a practical application by creating three different sales agents, each powered by a unique model (Deep Seek, Gemini, and Llama 3 via Groq). This showcases how to build heterogeneous agent systems to leverage the unique strengths or cost-benefits of different models for different tasks.
- **Structured Outputs**: This is introduced as a method to force an agent to return data in a specific object format (e.g., JSON with predefined fields) rather than unstructured text. This is critical for creating predictable and reliable data processing pipelines where agent outputs are fed into other software components.
- **Guardrails**: This concept is presented as a crucial feature for controlling the inputs and outputs of an agent system. Guardrails act as a safety layer to validate information, enforce rules, and prevent undesirable behavior, which is vital for production-ready applications.

### Conceptual Understanding
- **Using Non-OpenAI Models via Compatible Endpoints**
    1.  **Why is this concept important?** It decouples your agent logic from a single model provider (i.e., OpenAI). This flexibility prevents vendor lock-in and allows developers to optimize their systems by choosing the best model for a specific task based on cost, speed, or unique capabilities (e.g., using a fast, cheap model for summarization and a powerful, expensive model for complex reasoning).
    2.  **How does it connect to real-world tasks?** In a production environment, a data science team could build a research agent where a "manager" agent (using GPT-4) delegates literature search tasks to an agent running on Perplexity for its search focus and data extraction tasks to another agent using a fine-tuned open-source model for cost efficiency. The OpenAI Agents SDK orchestrates this entire workflow seamlessly.
    3.  **Which related techniques should be studied alongside this concept?** You should explore **API wrappers**, **model routing** (dynamically choosing a model based on the query), and **structured data validation libraries** like Pydantic, which are used to define the "structured outputs" mentioned in the tutorial.

### Reflective Questions
1.  **Application:** Which specific dataset or project could benefit from this multi-model concept? Provide a one-sentence explanation.
    - *Answer*: A customer support chatbot could use this by routing simple queries (e.g., "What are your hours?") to a fast, cheap model like Llama 3 on Groq, while escalating complex, emotional complaints to a more sophisticated model like Claude 3 Opus to ensure a high-quality, empathetic response.

2.  **Teaching:** How would you explain using a non-OpenAI model via a compatible endpoint to a junior colleague, using one concrete example? Keep the answer under two sentences.
    - *Answer*: Think of the SDK as a universal remote that's pre-programmed for an OpenAI TV. To control a different brand like a Gemini TV, you just need to enter its specific address (`base_url`) and access code (`api_key`) into the remote's settings.

3.  **Extension:** What related technique should you explore next, and why?
    - *Answer*: The next logical step is to master **structured outputs** using a library like Pydantic, because once you have multiple models producing information, you need a reliable way to force their responses into a consistent, machine-readable format for further processing.

# Day 3 - Implementing Guardrails & Structured Outputs for Robust AI Agent Systems

### Summary
This session analyzes the non-deterministic behavior of autonomous agents, demonstrating how a multi-agent system can fall into an infinite loop and highlighting the SDK's use of asynchronous execution for performance. It then introduces two powerful features for building robust applications: structured outputs, which force agents to return data in a predefined format using Pydantic classes, and guardrails, which act as LLM-powered validators to secure the inputs and outputs of the agent workflow. The tutorial also addresses a practical limitation: models without an OpenAI-compatible endpoint, like Anthropic's Claude, require workarounds such as proxy services (e.g., OpenRouter) to be integrated.

### Highlights
- **Agent Instability and Debugging**: The session begins by diagnosing a failed agent run that entered a loop, repeatedly calling the same tools. This serves as a practical example of the non-deterministic nature of LLM-based agents and a caution against giving them instructions that permit infinite retries without clear exit criteria.
- **Asynchronous Parallel Execution**: The debug trace reveals that the SDK executes I/O-bound tasks, like API calls to different models (Deep Seek, Gemini, Llama), in parallel. This asynchronous behavior is a core feature that significantly improves the performance of multi-agent systems.
- **Integrating Non-Compatible Models (Anthropic/Claude)**: It is clarified that models lacking a direct OpenAI-compatible API, such as Anthropic's Claude, cannot be used out-of-the-box. The recommended workaround is to use a third-party provider like OpenRouter, which offers a compatible endpoint that proxies requests to Claude.
- **Structured Outputs with Pydantic**: A key feature introduced is forcing an agent to produce a structured output instead of plain text. By defining a Pydantic class (e.g., `NameCheckOutput`) and assigning it to the agent's `output_type` parameter, the agent is compelled to return a populated object that conforms to the specified schema, making its output reliable and machine-readable.
- **Guardrails for Input/Output Control**: Guardrails are functions that act as safety checks at the entry and exit points of an agent workflow. An `@input_guardrail` runs before the first agent, and an `@output_guardrail` runs after the last, allowing you to validate data and prevent inappropriate content from being processed or shown to the user.
- **LLM-Powered Guardrails**: The most powerful aspect of guardrails is that they can themselves be agents. The example demonstrates creating a `guardrail_agent` that uses an LLM to perform a nuanced check (detecting if a personal name is in a message), which is far more sophisticated than a simple keyword filter.
- **Guardrail Implementation**: A guardrail is an `async` function that returns a `GuardrailFunctionOutput` object. The validation logic determines the value of the `tripwire_triggered` boolean in this object; if `True`, the guardrail fails, and the agent workflow is halted.

### Conceptual Understanding
- **Structured Outputs with Pydantic**
    1.  **Why is this concept important?** It enforces reliability and predictability in agent outputs. Instead of dealing with the variance of natural language text, your application receives a clean, validated data object, which eliminates the need for complex parsing and dramatically reduces the risk of downstream errors.
    2.  **How does it connect to real-world tasks?** An agent designed to extract details from an invoice could be forced to return an object with strictly typed fields like `{"invoice_id": int, "due_date": "YYYY-MM-DD", "total_amount": float}`. This guarantees the data is in the correct format to be inserted into a database or used in an accounting system without manual intervention.
    3.  **Which related techniques should be studied alongside this concept?** Familiarity with **data serialization** (e.g., JSON), **data validation libraries** (Pydantic is shown, but others like Marshmallow exist), and **API schema design** (like OpenAPI/Swagger) is highly relevant.

- **LLM-Powered Guardrails**
    1.  **Why is this concept important?** They create a dynamic, context-aware safety layer that is superior to static, rule-based checks. By leveraging an LLM, a guardrail can understand nuance, user intent, and context to block harmful inputs (like prompt injections) or prevent the generation of unsafe outputs (like giving financial advice).
    2.  **How does it connect to real-world tasks?** A customer service bot for a bank could have an input guardrail that uses an LLM to detect and block any user requests containing personally identifiable information (PII) like account numbers. An output guardrail could similarly use an LLM to verify that the bot's response does not contain any sensitive customer data before sending it.
    3.  **Which related techniques should be studied alongside this concept?** This connects directly to the fields of **AI Safety and Alignment**, **prompt engineering** (for creating effective guardrail prompts), **content moderation systems**, and **cybersecurity concepts** like defending against prompt injection attacks.

### Reflective Questions
1.  **Application:** Which specific dataset or project could benefit from a structured output? Provide a one-sentence explanation.
    - *Answer*: A resume parsing project would benefit immensely by forcing an agent to return a `Candidate` object with fields for `name`, `contact_info`, `work_experience`, and `skills`, ensuring all parsed resumes are standardized for easy querying and comparison.

2.  **Teaching:** How would you explain an LLM-powered guardrail to a non-technical manager?
    - *Answer*: "It's like adding a smart compliance officer to our AI's workflow. Before the AI acts on a request, this officer reads it to ensure it's appropriate and safe, and then it reviews the AI's answer to make sure it follows our company policies before it's sent to the user."

3.  **Extension:** What is a potential drawback of using an LLM-powered guardrail, and how might you mitigate it?
    - *Answer*: The primary drawbacks are increased latency and cost, as you're making an extra LLM call for each validation. To mitigate this, you could use a much faster, cheaper model for the guardrail task or implement a hybrid approach where simple, fast checks are performed first, and the LLM guardrail is only invoked for more complex or high-risk inputs.

# Day 3- AI Safety in Practice: Implementing Guardrails for LLM Agent Applications

### Summary
This session provides a practical demonstration of implementing an input guardrail within the OpenAI Agents SDK to enforce data safety. By running an agent with a prompt containing a forbidden personal name, the guardrail successfully identifies the name, triggers a tripwire, and halts execution by raising an exception, which is the intended behavior. The same agent then successfully processes a compliant prompt, illustrating how guardrails serve as a crucial tool for validating user input and preventing the misuse of personally identifiable information (PII) in automated, agent-driven applications.

### Highlights
- **Practical Guardrail Implementation**: A `CarefulSalesManager` agent is configured with an `input_guardrail`. This guardrail's function is to detect personal names in the initial prompt.
- **Triggering the Guardrail**: When the agent receives a prompt containing the name "Alice," the LLM-powered guardrail correctly identifies it. This action triggers the "tripwire," which is the mechanism for flagging a violation.
- **Exceptions as Control Flow**: A triggered guardrail intentionally causes the program to raise a `GuardrailInput[GuardrailTriggered]` exception. This is not a bug but the designed feature to immediately stop the workflow and prevent the agent from processing invalid or unsafe data.
- **Traceability for Debugging**: The execution trace is shown to be a valuable tool for understanding why a guardrail failed. The trace clearly displays the output from the `guardrail_agent`, showing that it identified the name and set `is_name_in_message` to `true`, which led to the exception.
- **Successful Validation Run**: The demonstration is repeated with a compliant prompt ("from the Head of Business Development"). In this case, the guardrail validates the input, the tripwire is not triggered, no exception is raised, and the agent workflow proceeds to completion, successfully generating an email.
- **Future Enhancements**: The session concludes by encouraging further experimentation, such as adding more complex guardrails (e.g., an output guardrail to check for outdated copyright years) and converting the final email output into a structured Pydantic object for more robust and reliable data handling.

### Reflective Questions
1.  **Application:** Which specific dataset or project could benefit from this guardrail concept? Provide a one-sentence explanation.
    - *Answer*: A medical chatbot project could use an input guardrail to detect and block any queries that ask for a diagnosis, thereby preventing the AI from giving unqualified medical advice and ensuring it operates within its safety guidelines.
2.  **Teaching:** How would you explain to a junior colleague why a program crash (an exception) from a guardrail is a good thing?
    - *Answer*: You can explain it as a safety feature, like a circuit breaker in a house. The exception isn't a random error; it's a deliberate action to immediately stop the system to prevent a bigger problem, like processing sensitive data or generating a harmful response.
